In [57]:
import numpy as np
import cv2
from PIL import Image,ImageEnhance
from imutils.perspective import four_point_transform
import cv2
import os

In [58]:
def auto_canny(image, sigma=0.4):
	# compute the median of the single channel pixel intensities
	v = np.median(image)
	# apply automatic Canny edge detection using the computed median
	lower = int(max(0, (1.0 - sigma) * v))
	upper = int(min(255, (1.0 + sigma) * v))
	edged = cv2.Canny(image, lower, upper)
	# return the edged image
	return edged

In [59]:
pathImage = r"data\omr_line.v2i.yolov7pytorch\train\images\310377047_465921045601836_3357657464690243685_n_jpg.rf.32b8bfbd56bde205a8bdec2fa9460b1a.jpg"

img_ori=Image.open(pathImage)
contrast_enhancer = ImageEnhance.Contrast(img_ori)
pil_enhanced_image = contrast_enhancer.enhance(2)
enhanced_image = np.asarray(pil_enhanced_image)
con = np.zeros_like(img_ori)
kernel = np.ones((5,5),np.uint8)
image_oricoppy=enhanced_image.copy()
enhanced_image = cv2.morphologyEx(enhanced_image, cv2.MORPH_CLOSE, kernel, iterations= 3)


imgGray = cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2GRAY)
imgBlur = cv2.GaussianBlur(imgGray, (5, 5), 1)
imgThreshold = auto_canny(imgBlur)
img = Image.fromarray(np.uint8(imgThreshold))
img.show()
kernel = np.ones((5, 5))
imgDial = cv2.dilate(imgThreshold, kernel, iterations=2)
imgThreshold = cv2.erode(imgDial, kernel, iterations=1)
img2 = Image.fromarray(np.uint8(imgThreshold))
img2.show()
contours, hierarchy = cv2.findContours(imgThreshold, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
page = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
#con = cv2.drawContours(con, page, -1, (0, 255, 255), 3)
for contour in contours:
    peri = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.05 * peri, True)
    if len(approx) == 4:
        doc_cnts = approx
        break
warped = four_point_transform(image_oricoppy, doc_cnts.reshape(4, 2))
img3 = Image.fromarray(np.uint8(warped))
img3.show()